# Bubble Map

This map shows the Bavarian focus of the data set.

In [1]:
'''
Reading pickled files
'''

import pandas as pd

person_dict = pd.read_pickle('./pickled/person_dict.p')
attr_df = pd.read_pickle('./pickled/dataframed_attributes.p')

/home/lyuba/plotly/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/lyuba/plotly/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
'''
Converting the place coordinates into a DataFrame.
'''

columns=['place', 'lat', 'lon', 'code']
aux_dfs = [pd.DataFrame(person_dict[person_id]['mapPlaces'], columns=columns) for person_id in person_dict]
    
df = pd.concat(aux_dfs, ignore_index=True)

In [3]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

mapbox_access_token = 'pk.eyJ1IjoibHl1YmEtZGltaXRyb3ZhIiwiYSI6ImNqbG0xdmdxdDEwYXczd25kODI5cHc2YXUifQ.8t0m5MoJW7cBkLQwNBCtAw'
init_notebook_mode(connected=True)

def create_map(df, place_code='3.1.'):
    """
    Outputs an interactive map of the most important places according to their frequency as people's main places
    (Hauptwirkungsort; Code: 3.1.) in the data set.
    """
    
    df = df[df.code == place_code]
    
    reduced = df.groupby(columns).size().reset_index().rename(columns={0:'count'})
    
    site_lat = reduced.lat
    site_lon = reduced.lon
    locations_name = reduced.place
    sizes = reduced['count']
    
    
    data = [
        go.Scattermapbox(
            lat=site_lat,
            lon=site_lon,
            mode='markers',
            marker=dict(
                size=sizes,
                sizemin=2,
                sizeref=15,
                sizemode='diameter',
                color=sizes,
                colorscale=[[0, 'blue'], [1, 'red']],
                opacity=0.7,
                colorbar=dict(
                    title='Number of mentions'
                )
            ),
            line=dict(
                width=1,
                color='black'),
            text=[', '.join([name, str(freq)]) for name, freq in zip(locations_name, sizes)],
            #text=locations_name,
            hoverinfo='text'
        )]

    layout = go.Layout(
        title='Main Places',
        autosize=True,
        hovermode='closest',
        showlegend=False,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(
                lat=50,
                lon=20
            ),
            pitch=0,
            zoom=2.5,
            style='light'
        ),
    )

    fig = dict(data=data, layout=layout)

    return iplot(fig, filename='jupyter-map')

In [4]:
create_map(df)